In [1]:
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine
import os

dotenv_path = '../.env' 
load_dotenv(dotenv_path=dotenv_path)

db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_name = os.getenv('DB_NAME')
db_connection_str = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
db_engine = create_engine(db_connection_str)

sql_query = "SELECT * FROM public.hyundai_segment_purchases;"

df = pd.read_sql(sql_query, db_engine)
df.head()

,Id,CarType,Manufacturer,Model,Age,Gender,Satisfaction,Review
0,41,SUV,현대,코나,60,남성,0.5,최악입니다. 다른 차량에서는 찾아볼 수 없는 휀다쪽 심한 단차로 인해 매일 스트레스...
1,42,중형,현대,쏘나타 디 엣지,30,남성,3.5,십년 이전 쏘나타 타다가 요번에 나온 차량 샀는데 신세계입니다. 우선은 디자인적으로...
2,43,대형,현대,그랜저 하이브리드,40,남성,5.0,아반떼MD를 약 14년간 오랜시간 별탈없이 타고 나이가 지긋하게 든 시점에서 그랜저...
3,44,중형,현대,쏘나타 디 엣지 하이브리드,50,남성,4.0,승차감.디자인.실내공간.연비효율.성능연에서 2년전 구입했던DN-8보다 월등히 탁월하...
4,45,SUV,현대,싼타페 하이브리드,50,남성,5.0,13년간 잘 타던 2010년형 아반떼md차량을 타다가 이번에 2025싼타페하이브리드...


In [ ]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine
from openai import OpenAI

load_dotenv()

class CarReviewAnalyzer:
    def __init__(self, db_config):
        self.engine = create_engine(
            f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['name']}"
        )
        self.client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

    def _top_model(self, age, gender):
        q = f'''
        SELECT "Model", COUNT(*) AS cnt
        FROM public.hyundai_segment_purchases
        WHERE "Age" = {age} AND "Gender" = '{gender}'
        GROUP BY "Model"
        ORDER BY cnt DESC
        LIMIT 1;
        '''
        df = pd.read_sql(q, self.engine)
        if df.empty:
            return None
        return df.iloc[0]['Model']

    def _avg_satisfaction(self, age, gender, model):
        q = f'''
        SELECT AVG("Satisfaction") AS avg_sat, COUNT(*) AS n
        FROM public.hyundai_segment_purchases
        WHERE "Age" = {age} AND "Gender" = '{gender}' AND "Model" = '{model}';
        '''
        df = pd.read_sql(q, self.engine)
        if df.empty or pd.isna(df.iloc[0]['avg_sat']):
            return None, 0
        # 이 부분을 round() 함수를 사용하여 반올림
        return round(float(df.iloc[0]['avg_sat']), 2), int(df.iloc[0]['n'])

    def _fetch_reviews(self, model, limit=200):
        q = f'''
        SELECT "Review"
        FROM public.hyundai_segment_purchases
        WHERE "Model" = '{model}'
        AND "Review" IS NOT NULL
        LIMIT {limit};
        '''
        df = pd.read_sql(q, self.engine)
        reviews = [r.strip() for r in df['Review'].dropna().astype(str).tolist() if r.strip()]
        return reviews

    def _llm_summarize(self, model, reviews):
        if not reviews:
            return {
                "summary_korean": f"{model}에 대한 리뷰가 충분하지 않습니다.",
                "overall_sentiment": {"label": "neutral", "score": 0.0},
                "top_aspects": [],
                "pros": [],
                "cons": [],
                "representative_quotes": []
            }
        sample = reviews[:120]
        joined = "\n---\n".join(sample)
        prompt = f"""
다음은 자동차 모델 '{model}'에 대한 실제 구매자 리뷰 일부입니다.
리뷰를 읽고 JSON으로만 답하세요.

요구 형식:
{{
  "summary_korean": "자연스러운 리뷰형식으로 4~6문장 요약. 실내공간에 관한 리뷰를 할때는 적재공간(ex. 골프백 수납이 몇개까지 가능한지 구체적인 비유)과 2열(혹은 3열 공간)에 대한 리뷰도 같이 언급할것",
  "overall_sentiment": {{"label": "positive|neutral|negative", "score": 0.0~1.0}},
  "top_aspects": [{{"name": "항목명", "sentiment": "pos|neu|neg"}} ... top_aspects 5개 고정],
  "pros": ["장점 요약(한 문장)", "..."],
  "cons": ["단점 요약(한 문장)", "..."],
  "representative_quotes": ["리뷰에서 짧은 인용 1~2개"]
}}

리뷰:
{joined}
"""
        resp = self.client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "너는 차량구매를 하려는 사용자에게 해당 모델의 장단점을 알려주는 분석가야"},
                {"role": "user", "content": prompt}
            ],
            temperature=0.2,
            max_tokens=800
        )
        txt = resp.choices[0].message.content.strip()
        try:
            data = json.loads(txt)

            sentiment_map = {
                "pos": "만족도 높음",
                "neu": "사용자마다 호불호가 갈림",
                "neg": "다소 아쉬움"
            }
            
            if "top_aspects" in data:
                for item in data["top_aspects"]:
                    sentiment_code = item.get("sentiment")
                    if sentiment_code in sentiment_map:
                        item["sentiment"] = sentiment_map[sentiment_code]
                        
        except Exception:
            data = {"summary_korean": txt[:600], "overall_sentiment": {"label": "neutral", "score": 0.0},
                    "top_aspects": [], "pros": [], "cons": [], "representative_quotes": []}
        return data

    def analyze(self, age, gender):
        model = self._top_model(age, gender)
        if not model:
            return {"error": "해당 조건의 구매 데이터가 없습니다."}
        avg_sat, n = self._avg_satisfaction(age, gender, model)
        reviews = self._fetch_reviews(model)
        llm = self._llm_summarize(model, reviews)
        return {
            "age": age,
            "gender": gender,
            "model": model,
            "avg_satisfaction": avg_sat,
            "sample_count": n,
            "llm_summary": llm
        }

if __name__ == "__main__":
    db_config = {
        'host': os.getenv('DB_HOST', 'localhost'),
        'port': os.getenv('DB_PORT', '5432'),
        'user': os.getenv('DB_USER', 'postgres'),
        'password': os.getenv('DB_PASSWORD', ''),
        'name': os.getenv('DB_NAME', 'carfin')
    }
    analyzer = CarReviewAnalyzer(db_config)
    result = analyzer.analyze(age=40, gender='남성')
    print(json.dumps(result, ensure_ascii=False, indent=2))

{
  "age": 40,
  "gender": "남성",
  "model": "싼타페 하이브리드",
  "avg_satisfaction": 4.93,
  "sample_count": 249,
  "llm_summary": {
    "summary_korean": "싼타페 하이브리드는 넓은 실내 공간과 뛰어난 연비로 많은 가족들에게 사랑받고 있습니다. 2열 좌석은 넉넉하여 아이들과 함께 여행할 때 편안함을 제공합니다. 트렁크 공간도 충분히 넓어 캠핑이나 장거리 여행 시 짐을 실기에 용이합니다. 하이브리드 모델답게 조용한 주행과 높은 연비는 운전의 즐거움을 더해줍니다. 디자인에 대한 호불호가 있지만, 많은 사용자들이 실용성과 편의성을 높이 평가하고 있습니다.",
    "overall_sentiment": {
      "label": "positive",
      "score": 0.95
    },
    "top_aspects": [
      {
        "name": "실내 공간",
        "sentiment": "만족도 높음"
      },
      {
        "name": "연비",
        "sentiment": "만족도 높음"
      },
      {
        "name": "승차감",
        "sentiment": "만족도 높음"
      },
      {
        "name": "디자인",
        "sentiment": "사용자마다 호불호가 갈림"
      },
      {
        "name": "편의 기능",
        "sentiment": "만족도 높음"
      }
    ],
    "pros": [
      "넓은 실내 공간으로 가족 여행에 적합하다.",
      "하이브리드 모델로 연비가 뛰어나고 조용한 주행이 가능하다.",
      "트렁크 공간이 넉넉하여 짐을 실기에 용이하다.",
      "최신 편의 기능이 잘 갖춰져 있어 운전이 편리하